In [63]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
wine_df = pd.read_csv('/Users/tobiaseckers/code/ArjanAngenent/VinoDine/raw_data/Cleaned_Full_100K_wines.csv')
wine_df.head()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages,Pork,RichFish,Shellfish,Beef,Barbecue,Codfish,Pasta,Pizza,Cheese,Lamb,Poultry,SweetDessert,GameMeat,Veal,SpicyFood,MaturatedCheese,HardCheese,Vegetarian,Appetizer,Snack,Fish,CitricDessert,Mushrooms,SoftCheese,Grilled,Soufflé,CuredMeat,Seafood,Risotto,Medium-curedCheese,Salad,LeanFish,FruitDessert,Dessert,Meat,Fruit,BlueCheese,Chicken,MildCheese,Chocolate,Ham,Cake,Yakissoba,Sashimi,CurryChicken,Sushi,BakedPotato,GoatCheese,Beans,Lasagna,EggplantParmigiana,Aperitif,Chestnut,FrenchFries,TomatoDishes,DriedFruits,SpicedFruitCake,LightStews,Duck,AsianFood,Cream,Cookies,ColdCuts,Roast,Tagliatelle,Paella,Muscat/Moscato,Ancellotta,CabernetSauvignon,Merlot,Muscat/MoscatoBianco,TourigaNacional,Syrah/Shiraz,ArintodeBucelas,Loureiro,Trajadura,TintaBarroca,TintaAmarela,TourigaFranca,TintaRoriz,TintoCão,Bordô,Tannat,Malvasia,Nebbiolo,Malbec,Chardonnay,Riesling,PinotNoir,Glera/Prosecco,Tempranillo,CabernetFranc,AlicanteBouschet,Sangiovese,PetitVerdot,Gewürztraminer,Sémillon,Viognier,Muscat/MoscatoGiallo,MuscatofHamburg,MuscatBlanc,Arinarnoa,SauvignonBlanc,GamayNoir,CheninBlanc,Verdejo,Teroldego,Marselan,UgniBlanc,Grenache,Barbera,Gouveio,PinotGrigio,MuscatofAlexandria,Seibel,Concord,Isabella,Pinotage,RieslingItálico,Torrontés,Egiodola,Carmenère,Johannisberg,Trebbiano,Peverella,PinotBlanc,PinotGris,PinotMeunier,Muscat/Muscatel,Alvarinho,Aragonez,Pecorino,Flora,Albariño,Lagrein,Couderc,PinotNero,Rebo,Refosco,RefoscodalPeduncoloRosso,Bonarda,Montepulciano,MalvasiaFina,Niagara,NiagaraBlanc,Lorena,TrebbianoToscano,Muscadelle,Corvina,Rondinella,Molinara,Fragolino,Roussanne,Marsanne,Vermentino,Violeta,TintoFino,Parellada,Bacchus,Kadarka,Arriloba,Mavrud,Vitovska,Carignan/Cariñena,MalvasiadiCandia,Trincadeira,Castelão,TintaMiúda,Camarate,AlfrocheiroPreto,Arinto,AzalBranco,TintadeToro,Sousão,Baga,AntãoVaz,Vinhão,Caladoc,Rufete,CôdegadeLarinho,Viosinho,Moreto,Encruzado,CercealBranco,Bical,Jaen,Pederna,Rabigato,Roupeiro,Perrum,TintaCaiada,Verdelho,PretoMartinho,FernãoPires,Kékfrankos,GrandNoir,Tibouren,TintaMadeira,Garnacha,Lacrima,Avesso,MuscatofSetúbal,RabodeOvelha,Treixadura,Vital,Cinsault,BastardoMagarachsky,TintaFrancisca,Tamarez,Síria,Marufo,PetitManseng,DonzelinhoBranco,Jampal,Sercial,MalvasiaNera,DonzelinhoTinto,MariaGomes,Terrantez,ArintodosAçores,DonaBlanca,VidalBlanc,Muscat/MoscatelGalego,Borraçal,Espadeiro,Muscat/MoscatelRoxo,Alvarelhão,Pedral,Mourvedre,Verdello,ÁguaSanta,PetiteSirah,AssarioBranco,AzalTinto,FonteCal,AlbillodeAlbacete,Batoca,GouveioReal,Mencia,BoalBranco,TintaCarvalha,Bual,SearaNova,AspiranBouschet,Diagalves,TintaNegraMole,Palomino,PedroXimenez,Mourisco,Ramisco,Padeiro,BlauerPortugieser,GrünerVeltliner,Primitivo,Chasselas,Folgasao,Trousseau,Cornifesto,Manteudo,AlicanteGanzin,ListanNegro,CaiñoTinto,Noble,Syriki,CabernetRuby,Arnsburger,Tintilla,PlavacMali,TintadelPais,Sercialinho,Bobal,Barcelo,Macabeo,Dornfelder,Chambourcin,CabernetDorsa,Rivaner,Silvaner/Sylvaner,Müller-Thurgau,Spätburgunder,Kerner,Weissburgunder,Grauburgunder,Scheurebe,Blaufränkisch,Lemberger,Ortega,Huxelrebe,Regent,Schwarzriesling,Zweigelt,Frühburgunder,GelberMuskateller,St.Laurent,Morio-Muskat,CabernetMitos,Rieslaner,SeyvalBlanc,PinotAuxerrois,CabernetCubin,Domina,OrleansGelb,Trollinger,PiccolaNera,Colombard,Acolon,Gutedel,Traminer,CabernetBlanc,SouvignierGris,MuscatGolden,Albalonga,MuscatOttonel,Monastrell,Dunkelfelder,Johanniter,FuméBlanc,Graciano,Rotberger,GrenacheBlanc,GrosManseng,Muscardin,GrenacheGris,Grolleau,FolleBlanche,Counoise,Aligoté,Aglianico,PiquepoulBlanc,Clairette,Picardan,MelondeBourgogne,Rolle/Rollo,Manseng,Zinfandel,Bourboulenc,PicpoulBlanc,SauvignonGris,PetitCourbu,Nielluccio,Sciacarello,Cannonau,MauzacBlanc,Chasan,PetitePearl,FerServadou,Vaccareze,SavagninBlanc,Poulsard,Terret,GarnachaTinta,Duras,Negrette,Abouriou,BacoNoir,BrunArgenté,Arbane,PetitMeslier,"""NerodAvola""",Riesel,"""PineauDAunis""",Pinenc,J

In [14]:
wine_df['Harmonize'].nunique()

900

In [17]:
wine_df['Grapes'].nunique()

7097

In [29]:
wine_df['Acidity'].unique()

array(['High', 'Medium', 'Low'], dtype=object)

# create categories for food and grapes

In [8]:
food_cats=['Pork',
 'RichFish',
 'Shellfish',
 'Beef',
 'Barbecue',
 'Codfish',
 'Pasta',
 'Pizza',
 'Cheese',
 'Lamb',
 'Poultry',
 'SweetDessert',
 'GameMeat',
 'Veal',
 'SpicyFood',
 'MaturatedCheese',
 'HardCheese',
 'Vegetarian',
 'Appetizer',
 'Snack',
 'Fish',
 'CitricDessert',
 'Mushrooms',
 'SoftCheese',
 'Grilled',
 'Soufflé',
 'CuredMeat',
 'Seafood',
 'Risotto',
 'Medium-curedCheese',
 'Salad',
 'LeanFish',
 'FruitDessert',
 'Dessert',
 'Meat',
 'Fruit',
 'BlueCheese',
 'Chicken',
 'MildCheese',
 'Chocolate',
 'Ham',
 'Cake',
 'Yakissoba',
 'Sashimi',
 'CurryChicken',
 'Sushi',
 'BakedPotato',
 'GoatCheese',
 'Beans',
 'Lasagna',
 'EggplantParmigiana',
 'Aperitif',
 'Chestnut',
 'FrenchFries',
 'TomatoDishes',
 'DriedFruits',
 'SpicedFruitCake',
 'LightStews',
 'Duck',
 'AsianFood',
 'Cream',
 'Cookies',
 'ColdCuts',
 'Roast',
 'Tagliatelle',
 'Paella']

In [10]:
grape_cats = ['Muscat/Moscato',
 'Ancellotta',
 'CabernetSauvignon',
 'Merlot',
 'Muscat/MoscatoBianco',
 'TourigaNacional',
 'Syrah/Shiraz',
 'ArintodeBucelas',
 'Loureiro',
 'Trajadura',
 'TintaBarroca',
 'TintaAmarela',
 'TourigaFranca',
 'TintaRoriz',
 'TintoCão',
 'Bordô',
 'Tannat',
 'Malvasia',
 'Nebbiolo',
 'Malbec',
 'Chardonnay',
 'Riesling',
 'PinotNoir',
 'Glera/Prosecco',
 'Tempranillo',
 'CabernetFranc',
 'AlicanteBouschet',
 'Sangiovese',
 'PetitVerdot',
 'Gewürztraminer',
 'Sémillon',
 'Viognier',
 'Muscat/MoscatoGiallo',
 'MuscatofHamburg',
 'MuscatBlanc',
 'Arinarnoa',
 'SauvignonBlanc',
 'GamayNoir',
 'CheninBlanc',
 'Verdejo',
 'Teroldego',
 'Marselan',
 'UgniBlanc',
 'Grenache',
 'Barbera',
 'Gouveio',
 'PinotGrigio',
 'MuscatofAlexandria',
 'Seibel',
 'Concord',
 'Isabella',
 'Pinotage',
 'RieslingItálico',
 'Torrontés',
 'Egiodola',
 'Carmenère',
 'Johannisberg',
 'Trebbiano',
 'Peverella',
 'PinotBlanc',
 'PinotGris',
 'PinotMeunier',
 'Muscat/Muscatel',
 'Alvarinho',
 'Aragonez',
 'Pecorino',
 'Flora',
 'Albariño',
 'Lagrein',
 'Couderc',
 'PinotNero',
 'Rebo',
 'Refosco',
 'RefoscodalPeduncoloRosso',
 'Bonarda',
 'Montepulciano',
 'MalvasiaFina',
 'Niagara',
 'NiagaraBlanc',
 'Lorena',
 'TrebbianoToscano',
 'Muscadelle',
 'Corvina',
 'Rondinella',
 'Molinara',
 'Fragolino',
 'Roussanne',
 'Marsanne',
 'Vermentino',
 'Violeta',
 'TintoFino',
 'Parellada',
 'Bacchus',
 'Kadarka',
 'Arriloba',
 'Mavrud',
 'Vitovska',
 'Carignan/Cariñena',
 'MalvasiadiCandia',
 'Trincadeira',
 'Castelão',
 'TintaMiúda',
 'Camarate',
 'AlfrocheiroPreto',
 'Arinto',
 'AzalBranco',
 'TintadeToro',
 'Sousão',
 'Baga',
 'AntãoVaz',
 'Vinhão',
 'Caladoc',
 'Rufete',
 'CôdegadeLarinho',
 'Viosinho',
 'Moreto',
 'Encruzado',
 'CercealBranco',
 'Bical',
 'Jaen',
 'Pederna',
 'Rabigato',
 'Roupeiro',
 'Perrum',
 'TintaCaiada',
 'Verdelho',
 'PretoMartinho',
 'FernãoPires',
 'Kékfrankos',
 'GrandNoir',
 'Tibouren',
 'TintaMadeira',
 'Garnacha',
 'Lacrima',
 'Avesso',
 'MuscatofSetúbal',
 'RabodeOvelha',
 'Treixadura',
 'Vital',
 'Cinsault',
 'BastardoMagarachsky',
 'TintaFrancisca',
 'Tamarez',
 'Síria',
 'Marufo',
 'PetitManseng',
 'DonzelinhoBranco',
 'Jampal',
 'Sercial',
 'MalvasiaNera',
 'DonzelinhoTinto',
 'MariaGomes',
 'Terrantez',
 'ArintodosAçores',
 'DonaBlanca',
 'VidalBlanc',
 'Muscat/MoscatelGalego',
 'Borraçal',
 'Espadeiro',
 'Muscat/MoscatelRoxo',
 'Alvarelhão',
 'Pedral',
 'Mourvedre',
 'Verdello',
 'ÁguaSanta',
 'PetiteSirah',
 'AssarioBranco',
 'AzalTinto',
 'FonteCal',
 'AlbillodeAlbacete',
 'Batoca',
 'GouveioReal',
 'Mencia',
 'BoalBranco',
 'TintaCarvalha',
 'Bual',
 'SearaNova',
 'AspiranBouschet',
 'Diagalves',
 'TintaNegraMole',
 'Palomino',
 'PedroXimenez',
 'Mourisco',
 'Ramisco',
 'Padeiro',
 'BlauerPortugieser',
 'GrünerVeltliner',
 'Primitivo',
 'Chasselas',
 'Folgasao',
 'Trousseau',
 'Cornifesto',
 'Manteudo',
 'AlicanteGanzin',
 'ListanNegro',
 'CaiñoTinto',
 'Noble',
 'Syriki',
 'CabernetRuby',
 'Arnsburger',
 'Tintilla',
 'PlavacMali',
 'TintadelPais',
 'Sercialinho',
 'Bobal',
 'Barcelo',
 'Macabeo',
 'Dornfelder',
 'Chambourcin',
 'CabernetDorsa',
 'Rivaner',
 'Silvaner/Sylvaner',
 'Müller-Thurgau',
 'Spätburgunder',
 'Kerner',
 'Weissburgunder',
 'Grauburgunder',
 'Scheurebe',
 'Blaufränkisch',
 'Lemberger',
 'Ortega',
 'Huxelrebe',
 'Regent',
 'Schwarzriesling',
 'Zweigelt',
 'Frühburgunder',
 'GelberMuskateller',
 'St.Laurent',
 'Morio-Muskat',
 'CabernetMitos',
 'Rieslaner',
 'SeyvalBlanc',
 'PinotAuxerrois',
 'CabernetCubin',
 'Domina',
 'OrleansGelb',
 'Trollinger',
 'PiccolaNera',
 'Colombard',
 'Acolon',
 'Gutedel',
 'Traminer',
 'CabernetBlanc',
 'SouvignierGris',
 'MuscatGolden',
 'Albalonga',
 'MuscatOttonel',
 'Monastrell',
 'Dunkelfelder',
 'Johanniter',
 'FuméBlanc',
 'Graciano',
 'Rotberger',
 'GrenacheBlanc',
 'GrosManseng',
 'Muscardin',
 'GrenacheGris',
 'Grolleau',
 'FolleBlanche',
 'Counoise',
 'Aligoté',
 'Aglianico',
 'PiquepoulBlanc',
 'Clairette',
 'Picardan',
 'MelondeBourgogne',
 'Rolle/Rollo',
 'Manseng',
 'Zinfandel',
 'Bourboulenc',
 'PicpoulBlanc',
 'SauvignonGris',
 'PetitCourbu',
 'Nielluccio',
 'Sciacarello',
 'Cannonau',
 'MauzacBlanc',
 'Chasan',
 'PetitePearl',
 'FerServadou',
 'Vaccareze',
 'SavagninBlanc',
 'Poulsard',
 'Terret',
 'GarnachaTinta',
 'Duras',
 'Negrette',
 'Abouriou',
 'BacoNoir',
 'BrunArgenté',
 'Arbane',
 'PetitMeslier',
 '"NerodAvola"',
 'Riesel',
 '"PineauDAunis"',
 'Pinenc',
 'Jacquère',
 'MuscatNoir',
 'Gringet',
 'Côt',
 'Cortese',
 'Lauzet',
 'Aramon',
 'Airen',
 'Vranac',
 'Sacy',
 'Altesse',
 'Corvinone',
 'MondeuseNoire',
 'CabernetJura',
 'Braucol',
 'Garganega',
 '"LoindelOeil"',
 'Marmajuelo',
 '"RoussetteDAyze"',
 'Saperavi',
 'MuscatofFrontignan',
 'BlackQueen',
 'SwensonWhite',
 'GrecoNero',
 'GarnachaBlanca',
 'Fiano',
 'Segalin',
 'Valdiguie',
 'Trepat',
 'RabosoPiave',
 'Picolit',
 'RoyaldeAlloza',
 'Assyrtiko',
 'Vernaccia',
 'Chenanson',
 'Fredonia',
 '"LendelEl"',
 'GamayTeinturierdeBouze',
 'Abrostine',
 'Perle',
 'Öküzgözü',
 'PosipBijeli',
 'Oseleta',
 'Negroamaro',
 'Croatina',
 'CanaioloNero',
 'Ciliegiolo',
 'NegraraVeronese',
 'Rossignola',
 'Ansonica',
 'Frappato',
 'Verdicchio',
 'NerelloMascalese',
 'Susumaniello',
 'NerodiTroia',
 'Lambrusco',
 'VerduzzoFriulano',
 'Greco',
 'Falanghina',
 'Arneis',
 'Sagrantino',
 'Grechetto',
 'Procanico',
 'Ruché',
 'Grecanico',
 'Marzemino',
 'Groppello',
 'Cesanese',
 'TocaiFriulano',
 'TocaiItalico',
 'RibollaGialla',
 'Piedirosso',
 'CatarrattoBianco',
 'VernacciadiOristano',
 'Dolcetto',
 'Bovale',
 'Monica',
 'Ginestra',
 'Ripolo',
 'Fenile',
 'NegraraTrentino',
 'TrebbianoGiallo',
 'Freisa',
 'Albarossa',
 'ColorinodelValdarno',
 'NerelloCappuccio',
 'Zibibbo',
 'Brachetto',
 'Friulano/Sauvignonasse',
 'Bosco',
 'Sciascinoso',
 'Inzolia',
 'Pelaverga',
 'Gaglioppo',
 'Pascale',
 'Dindarella',
 'Morellino',
 'Roscetto',
 'Blauburgunder',
 'Schioppettino',
 'RieslingRenano',
 'Carricante',
 'Pugnitello',
 'Drupeggio',
 'Perricone',
 'Vespolina',
 'VernacciadiSanGimignano',
 'Albanella',
 'Timorasso',
 'PrugnoloGentile',
 'Grillo',
 'PetiteArvine',
 'Vernatsch',
 'Nocera',
 'Mammolo',
 'Ortrugo',
 'ValentinoNero',
 'Torbato',
 'Abbuoto',
 'FogliaTonda',
 'BianchettaTrevigiana',
 'Perera',
 'Nosiola',
 'Schiava',
 'Lampia',
 'Albana',
 'Biancame',
 'Pignoletto',
 'IncrocioManzoni',
 'NeroBuonodiCori',
 'Nuragus',
 'UvaRara',
 'Pigato',
 'Enantio',
 'Nascetta',
 'RabosoVeronese',
 'Passerina',
 'Verdiso',
 'Rossola',
 'Biancolella',
 'Coloraillo',
 'Erbaluce',
 '"TrebbianodAbruzzo"',
 'Cococciola',
 'Famoso',
 'Tintilia',
 'PolleraNera',
 'PallagrelloNero',
 'Casavecchia',
 'TintoreDiTramonti',
 'GrecoBianco',
 'Albarola',
 'ManzoniBianco',
 'Fumin',
 'MaglioccoCanino',
 'MalvasiadiLipari',
 'CorintoNero',
 'Pignolo',
 'MalvasiadelLazio',
 'Verdeca',
 'Nieddera',
 'Durella',
 'BombinoBianco',
 'PetitRouge',
 'Aleatico',
 'BombinoNero',
 'Barsaglina',
 'Blauburger',
 'VermentinoNero',
 'Nasco',
 'Casetta',
 'Cagnulari',
 'CanaioloBlanco',
 'Barbarossa',
 'Pagadebit',
 'Xarel-lo',
 'Vespaiola',
 'MalvasiaBiancaLunga',
 'Furmint',
 'Mayolet',
 'Rossese',
 'MalvasiaIstriana',
 'Pampanuto',
 'Grignolino',
 'Morenillo',
 'CodadiVolpeBianca',
 'Premetta',
 'TrebbianoRomagnolo',
 'Favorita',
 'Moradella',
 'SchiavaGrigia',
 'Pulcinculo',
 'Muscat/MoscatoRosa',
 'Tazzelenghe',
 'MalvasiadiSchierano',
 '"BiancodAlessano"',
 'CalabresediMontenuovo',
 'IrsaiOliver',
 'FalanghinaBeneventana',
 'MuscatYellow',
 'GambadiPernice',
 'Chatus',
 'Avanà',
 'Montuni',
 'Muscat/MoscatodiScanzo',
 'UghettadiCanneto',
 'GoldTraminer',
 'Ruggine',
 'MalboGentile',
 'ZinfandelWhite',
 'PallagrelloBianco',
 'AsprinioBianco',
 'Cornalin',
 'Guardavalle',
 'Prié',
 'Michet',
 'Bianca',
 'ViendeNus',
 'Solaris',
 'CabernetSeverny',
 'GrechettoRosso',
 'Bellone',
 'Turbiana',
 'Chardonel',
 'SchiavaGentile',
 'Centesiminio',
 'Malvar',
 'Pavana',
 'NerettaCuneese',
 'Braquet',
 'Aurore',
 'Terrano',
 'Reichensteiner',
 'Schönburger',
 'MadeleineAngevine',
 'Viura',
 'Siegerrebe',
 'Orion',
 'Phoenix',
 'Rondo',
 'Mazuelo',
 'Callet',
 'Samsó',
 'GarnachaTintorera',
 'MauzacNoir',
 'Godello',
 'HondarrabiZuri',
 'GarnachaPeluda',
 'MaturanaBlanca',
 'TempranilloBlanco',
 'MantoNegro',
 'Albillo',
 'Cencibel',
 'Moristel',
 'MaturanaTinta',
 'Prensal',
 'Sumoll',
 'Merseguera',
 'HondarrabiBeltza',
 'Muscat/MoscateldeGranoMenudo',
 'Parraleta',
 'PrietoPicudo',
 'Mandón',
 'CaiñoBlanco',
 'Zalema',
 'Mollard',
 'GarnachaRoja',
 'JuanGarcia',
 'UlldeLlebre',
 'AlbarínBlanco',
 'AlbilloMayor',
 'GiròBlanc',
 'PiquepoulNoir',
 'Gorgollasa',
 'Muscat/MoscatelloSelvatico',
 'Bruñal',
 'Vijiriega',
 'Romé',
 'SumollBlanc',
 'GarridoFino',
 'Rkatsiteli',
 'AlbilloCrimean',
 'KokurBely',
 'MuscatBlack',
 'Mataro',
 'País/Mission',
 'Dakapo',
 'CriollaGrande',
 'Bonamico',
 'Muscadine',
 'ShirokaMelnishka',
 'Bukettraube',
 'GrasadeCotnari',
 'Nouvelle',
 'Durif',
 'Crouchen',
 'HumagneBlanche',
 'Hanepoot',
 'Roobernet',
 'Cienna',
 'Frontenac',
 'Tarrango',
 'Sultaniye',
 'Ondenc',
 'MuscatOrange',
 'Charbono',
 'Rubired',
 'Symphony',
 'Sultana',
 'Norton',
 'CayugaWhite',
 'Vignoles',
 'GrosVerdot',
 'MuscatEarly',
 'Lenoir',
 'Marquette',
 'Steuben',
 'Sabrevois',
 'Chancellor',
 'DeChaunac',
 'Rougeon',
 'MarechalFoch',
 'Triomphe',
 'Melody',
 'Colobel',
 'Catawba',
 'Traminette',
 'CorotNoir',
 'Noiret',
 'Cynthiana',
 'Peloursin',
 'Cascade',
 'Millot-Foch',
 'LeonMillot',
 'VillardNoir',
 'LandotNoir',
 'MuscatValvin',
 'Edelweiss',
 'St.Croix',
 'Vincent',
 'Brianna',
 'Ehrenfelser',
 'ChardonnayMusqué',
 '"LAcadieBlanc"',
 'PetiteMilo',
 'MuscatBaileyA',
 'Koshu',
 'Malagouzia',
 'Bogazkere',
 'Xinomavro',
 'Moschofilero',
 'Hárslevelű',
 'Agiorgitiko',
 'Aidani',
 'Athiri',
 'Gaidouria',
 'Katsano',
 'Zéta',
 'Roditis',
 'Sideritis',
 'Welschriesling',
 'Amigne',
 'Melnik',
 'Savatiano',
 'FeteascaRegala',
 'Turan',
 'KrasnostopZolotovsky',
 'BăbeascăNeagră',
 'Limnio',
 'MalvazijaIstarska',
 'Sibirkovi',
 'HumagneRouge',
 'Vilana',
 'Debina',
 'KrasnostopAnapsky',
 'Krstac',
 'Smederevka',
 'MavroKalavritino',
 'Kangun',
 'Dimiat',
 'Mavrotragano',
 'Golubok',
 'TsimlyanskyCherny',
 'Kabar',
 'Areni',
 'Karmahyut',
 'Mandilaria',
 'Diolinoir',
 'Leanyka',
 'Zengö',
 'Rotgipfler',
 'Zierfandler',
 'Kotsifali',
 'FeteascaNeagra',
 'Kratosija',
 'Limniona',
 'MtsvaneKakhuri',
 'Neuburger',
 'Liatiko',
 'Mavrodafni',
 'FeteascaAlba',
 'Negoska',
 'Gamaret',
 'NegrudeDragasani',
 'Narince',
 'Xynisteri',
 'Mouhtaro',
 'Girgentina',
 'Tsolikouri',
 'OlaszRizling',
 'Hibernal',
 'Zelen',
 'TamaioasaRomaneasca',
 'EmeraldRiesling',
 'Mavro',
 'Dimrit',
 'MavroudiArachovis',
 'Prokupac',
 'Garanoir',
 'Argaman',
 'Dafni',
 'Khikhvi',
 'Kisi',
 'MisketVrachanski',
 'Thrapsathiri',
 'Karasakiz',
 'Robola',
 'Ruen',
 'Vidiano',
 'Emir',
 'Romeiko',
 'Maratheftiko',
 'Räuschling',
 'ModrýPortugal',
 'Ruländer',
 'Szürkebarát',
 'Lefkada',
 'RoterVeltliner',
 'MustoasadeMaderat',
 'Calkarasi',
 'Avgoustiatis',
 'Keratsuda',
 'MisketCherven',
 'Kydonitsa',
 'CabernetCortis',
 'GoruliMtsvane',
 'Chinuri',
 'Goldburger',
 'Menoir',
 'Muscat/MuskatMoravsky',
 'FruhroterVeltliner',
 'BudaiZöld',
 'BusuioacadeBohotin',
 'Galotta',
 'Rosetta',
 'Fokiano',
 'Viorica',
 'Morava',
 'Gigiac',
 'Roesler',
 'Moschomavro',
 'Alibernet',
 'Babić',
 'Kakotrygis',
 'Francusa',
 'Rubin',
 'Zlahtina',
 'Sarba',
 'KalecikKarasi',
 'Királyleányka',
 'Palava',
 'Kakhet',
 'OtskhanuriSapere',
 'Lafnetscha',
 'Ojaleshi',
 'BurgundMare',
 'Bronner',
 'Helios',
 'DzvelshaviObchuri',
 'Klevner',
 'Pamid']

In [16]:
len(grape_cats)

777

# create base model

In [47]:
# features to select: Type, Elaborate, Grapes, ABV, Body, Acidity
# target: harmonize
y = wine_df[food_cats]
X = wine_df[['Type', 'Elaborate', 'ABV', 'Body', 'Acidity'] + grape_cats]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)

# Create Ordinal Enoder for Acidity with 3 categories ['High', 'Medium', 'Low']
ordinal_encoder = OrdinalEncoder(categories=[['High', 'Medium', 'Low']])

# Create StandardScaler for ABV(numerical feeature)
standard_scaler = StandardScaler()

# fit_transform encoders and scalers to X_train

X_train[ohe.get_feature_names_out()] = ohe.fit_transform(X_train[['Type','Elaborate','Body']])

X_train['encoded_acidicity'] = ordinal_encoder.fit_transform(X_train[['Acidity']])

X_train['ABV'] = standard_scaler.fit_transform(X_train[['ABV']])

X_train.drop(columns=['Type','Elaborate','Body', 'Acidity'], inplace=True)

# transform X_test with encoders and scalers
X_test[ohe.get_feature_names_out()] = ohe.transform(X_test[['Type','Elaborate','Body']])

X_test['encoded_acidicity'] = ordinal_encoder.transform(X_test[['Acidity']])

X_test['ABV'] = standard_scaler.transform(X_test[['ABV']])

X_test.drop(columns=['Type','Elaborate','Body', 'Acidity'], inplace=True)

In [48]:
X_train.head()

,ABV,Muscat/Moscato,Ancellotta,CabernetSauvignon,Merlot,Muscat/MoscatoBianco,TourigaNacional,Syrah/Shiraz,ArintodeBucelas,Loureiro,Trajadura,TintaBarroca,TintaAmarela,TourigaFranca,TintaRoriz,TintoCão,Bordô,Tannat,Malvasia,Nebbiolo,Malbec,Chardonnay,Riesling,PinotNoir,Glera/Prosecco,Tempranillo,CabernetFranc,AlicanteBouschet,Sangiovese,PetitVerdot,Gewürztraminer,Sémillon,Viognier,Muscat/MoscatoGiallo,MuscatofHamburg,MuscatBlanc,Arinarnoa,SauvignonBlanc,GamayNoir,CheninBlanc,Verdejo,Teroldego,Marselan,UgniBlanc,Grenache,Barbera,Gouveio,PinotGrigio,MuscatofAlexandria,Seibel,Concord,Isabella,Pinotage,RieslingItálico,Torrontés,Egiodola,Carmenère,Johannisberg,Trebbiano,Peverella,PinotBlanc,PinotGris,PinotMeunier,Muscat/Muscatel,Alvarinho,Aragonez,Pecorino,Flora,Albariño,Lagrein,Couderc,PinotNero,Rebo,Refosco,RefoscodalPeduncoloRosso,Bonarda,Montepulciano,MalvasiaFina,Niagara,NiagaraBlanc,Lorena,TrebbianoToscano,Muscadelle,Corvina,Rondinella,Molinara,Fragolino,Roussanne,Marsanne,Vermentino,Violeta,TintoFino,Parellada,Bacchus,Kadarka,Arriloba,Mavrud,Vitovska,Carignan/Cariñena,MalvasiadiCandia,Trincadeira,Castelão,TintaMiúda,Camarate,AlfrocheiroPreto,Arinto,AzalBranco,TintadeToro,Sousão,Baga,AntãoVaz,Vinhão,Caladoc,Rufete,CôdegadeLarinho,Viosinho,Moreto,Encruzado,CercealBranco,Bical,Jaen,Pederna,Rabigato,Roupeiro,Perrum,TintaCaiada,Verdelho,PretoMartinho,FernãoPires,Kékfrankos,GrandNoir,Tibouren,TintaMadeira,Garnacha,Lacrima,Avesso,MuscatofSetúbal,RabodeOvelha,Treixadura,Vital,Cinsault,BastardoMagarachsky,TintaFrancisca,Tamarez,Síria,Marufo,PetitManseng,DonzelinhoBranco,Jampal,Sercial,MalvasiaNera,DonzelinhoTinto,MariaGomes,Terrantez,ArintodosAçores,DonaBlanca,VidalBlanc,Muscat/MoscatelGalego,Borraçal,Espadeiro,Muscat/MoscatelRoxo,Alvarelhão,Pedral,Mourvedre,Verdello,ÁguaSanta,PetiteSirah,AssarioBranco,AzalTinto,FonteCal,AlbillodeAlbacete,Batoca,GouveioReal,Mencia,BoalBranco,TintaCarvalha,Bual,SearaNova,AspiranBouschet,Diagalves,TintaNegraMole,Palomino,PedroXimenez,Mourisco,Ramisco,Padeiro,BlauerPortugieser,GrünerVeltliner,Primitivo,Chasselas,Folgasao,Trousseau,Cornifesto,Manteudo,AlicanteGanzin,ListanNegro,CaiñoTinto,Noble,Syriki,CabernetRuby,Arnsburger,Tintilla,PlavacMali,TintadelPais,Sercialinho,Bobal,Barcelo,Macabeo,Dornfelder,Chambourcin,CabernetDorsa,Rivaner,Silvaner/Sylvaner,Müller-Thurgau,Spätburgunder,Kerner,Weissburgunder,Grauburgunder,Scheurebe,Blaufränkisch,Lemberger,Ortega,Huxelrebe,Regent,Schwarzriesling,Zweigelt,Frühburgunder,GelberMuskateller,St.Laurent,Morio-Muskat,CabernetMitos,Rieslaner,SeyvalBlanc,PinotAuxerrois,CabernetCubin,Domina,OrleansGelb,Trollinger,PiccolaNera,Colombard,Acolon,Gutedel,Traminer,CabernetBlanc,SouvignierGris,MuscatGolden,Albalonga,MuscatOttonel,Monastrell,Dunkelfelder,Johanniter,FuméBlanc,Graciano,Rotberger,GrenacheBlanc,GrosManseng,Muscardin,GrenacheGris,Grolleau,FolleBlanche,Counoise,Aligoté,Aglianico,PiquepoulBlanc,Clairette,Picardan,MelondeBourgogne,Rolle/Rollo,Manseng,Zinfandel,Bourboulenc,PicpoulBlanc,SauvignonGris,PetitCourbu,Nielluccio,Sciacarello,Cannonau,MauzacBlanc,Chasan,PetitePearl,FerServadou,Vaccareze,SavagninBlanc,Poulsard,Terret,GarnachaTinta,Duras,Negrette,Abouriou,BacoNoir,BrunArgenté,Arbane,PetitMeslier,"""NerodAvola""",Riesel,"""PineauDAunis""",Pinenc,Jacquère,MuscatNoir,Gringet,Côt,Cortese,Lauzet,Aramon,Airen,Vranac,Sacy,Altesse,Corvinone,MondeuseNoire,CabernetJura,Braucol,Garganega,"""LoindelOeil""",Marmajuelo,"""RoussetteDAyze""",Saperavi,MuscatofFrontignan,BlackQueen,SwensonWhite,GrecoNero,GarnachaBlanca,Fiano,Segalin,Valdiguie,Trepat,RabosoPiave,Picolit,RoyaldeAlloza,Assyrtiko,Vernaccia,Chenanson,Fredonia,"""LendelEl""",GamayTeinturierdeBouze,Abrostine,Perle,Öküzgözü,PosipBijeli,Oseleta,Negroamaro,Croatina,CanaioloNero,Ciliegiolo,NegraraVeronese,Rossignola,Ansonica,Frappato,Verdicchio,NerelloMascalese,Susumaniello,NerodiTroia,Lambrusco,VerduzzoFriulano,Greco,Falanghina,Arneis,Sagrantino,Grechetto,Procanico,Ruché,Grecanico,Marzemino,Groppello,Cesanese,TocaiFriulano,Toca

In [64]:
model = SGDClassifier()
# Make it an Multilabel classifier
multilabel_classifier = MultiOutputClassifier(model, n_jobs=-1)

# Fit the data to the Multilabel classifier
multilabel_classifier = multilabel_classifier.fit(X_train, y_train)

ValueError: The number of classes has to be greater than one; got 1 class

In [55]:
cross_val_score(estimator=LinearSVC,X=X_train,y=y_train, cv=5, n_jobs=-1)

TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.